In [1]:
from building_depot import DataService, BDError
import json
import pandas as pd
import arrow

In [2]:
with open("config/ion_config.json", "r") as fp:
    configs = json.load(fp)
    username = configs['username']
    apikey = configs['api_key']
    base_url = configs['base_url']
    start_time = arrow.get(configs['start_time']).datetime
    end_time = arrow.get(configs['end_time']).datetime
datadir = "./data/"

In [7]:
def make_series(bd_data):
    time_list = list()
    value_list = list()
    for datum in data:
        time_list.append(datum.keys()[0])
        value_list.append(datum.values()[0])
    return pd.Series(index=time_list, data=value_list)

In [4]:
bd = DataService(base_url, apikey, username)
orig_metadata_df = pd.DataFrame.from_csv('metadata/ion.txt', )

In [5]:
srcid_dict = dict()
for uuid in orig_metadata_df['uuid']:
    data = bd.get_timeseries_datapoints(uuid, "PresentValue", start_time, end_time)['timeseries']
    if len(data)>1:
        sensor = bd.view_sensor(uuid)
        srcid_dict[uuid] = sensor['source_identifier']
        #make_series(data).to_csv(datadir+uuid+'.csv')

KeyboardInterrupt: 

In [6]:
pd.DataFrame.from_dict(data={"uuid":srcid_dict.keys(), "srcid":srcid_dict.values()}).to_csv("metadata/raw_ion_metadata.csv")
pd.DataFrame.from_dict(data={"uuid":srcid_dict.keys(), "srcid":srcid_dict.values()}).to_json("metadata/ion_metadata.json")

In [71]:
def unit_finder(srcid):
    if "BTU" in srcid:
        return "BTU_TH"
    elif "Power" in srcid:
        return "WATT"